In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
path = "../data"
data = pd.read_csv(f"{path}/joined_data.csv")
data['item_count'] = data['item_cnt_day']
data = data[['item_id', 'item_price', 'date_block_num', 'shop_id', "item_count", 'month']]

In [2]:
data.columns

Index(['item_id', 'item_price', 'date_block_num', 'shop_id', 'item_count',
       'month'],
      dtype='object')

In [3]:
target = data['item_count']
explanatory = data[['item_id', 'item_price', 'date_block_num', 'shop_id', 'month']]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    explanatory, target, test_size = 0.2, random_state = 42
)

In [5]:
X_train

,item_id,item_price,date_block_num,shop_id,month
1180918,3141,1130.0,7,35,8
173175,4421,299.0,0,50,1
2031652,6488,699.0,17,39,6
2352591,7220,1999.0,23,53,12
40397,4712,299.0,0,10,1
...,...,...,...,...,...
110268,22014,399.0,0,27,1
1692743,8093,1399.0,12,44,1
2356330,6911,599.0,23,50,12
2229084,687,600.0,21,16,10


In [6]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

ModuleNotFoundError: No module named 'imblearn'

In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 30)

clf.fit(X_train, y_train)

: 

In [8]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       0.14      0.02      0.04    274251
         2.0       0.11      0.01      0.03    109622
         3.0       0.09      0.01      0.02     55066
         4.0       0.09      0.01      0.02     32419
         5.0       0.08      0.01      0.03     20102
         6.0       0.08      0.02      0.03     13145
         7.0       0.09      0.02      0.04      8534
         8.0       0.05      0.01      0.02      5570
         9.0       0.06      0.02      0.03      3486
        10.0       0.02      0.01      0.01      2146
        11.0       0.01      0.01      0.01      1165
        12.0       0.00      0.00      0.00       824
        13.0       0.00      0.00      0.00       319
        14.0       0.00      0.00      0.00       172
        15.0       0.00      0.00      0.00        85
        16.0       0.00      0.00      0.00        27
        17.0       0.00    

/workspaces/Arquitectura/.arquivenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/workspaces/Arquitectura/.arquivenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/workspaces/Arquitectura/.arquivenv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} 

In [7]:
from xgboost import XGBClassifier
import numpy as np
model = XGBClassifier()
y_train_boost = np.array(y_train, dtype = int)
y_train_boost = y_train_boost - 1
model.fit(X_train, y_train_boost)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [10]:
predictions = model.predict(X_test)
y_test = np.array(y_test, dtype = int)
y_test -= 1
print(classification_report(y_test, predictions))


# ## AUC ROC 
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
from sklearn.metrics import roc_auc_score

# Calculate ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC: {roc_auc:.2f}")

              precision    recall  f1-score   support

           0       0.60      0.97      0.74    274251
           1       0.30      0.12      0.17    109622
           2       0.31      0.08      0.13     55066
           3       0.40      0.08      0.14     32419
           4       0.43      0.09      0.15     20102
           5       0.42      0.11      0.18     13145
           6       0.53      0.17      0.25      8534
           7       0.57      0.24      0.33      5570
           8       0.62      0.28      0.39      3486
           9       0.60      0.32      0.41      2146
          10       0.78      0.45      0.57      1165
          11       0.78      0.64      0.70       824
          12       0.94      0.86      0.90       319
          13       0.95      0.95      0.95       172
          14       1.00      0.88      0.94        85
          15       1.00      1.00      1.00        27
          16       1.00      1.00      1.00        22

    accuracy              

ValueError: multi_class must be in ('ovo', 'ovr')

In [12]:
y_pred_proba

array([0.21347773, 0.09640373, 0.2007733 , ..., 0.3129121 , 0.17256866,
       0.13088602], shape=(526955,), dtype=float32)